In [1]:
import os
import numpy as np
import time
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# --- Step 0: Start timer ---
start_time = time.time()

# --- Step 1: Set paths ---
home = os.path.expanduser('~')
file_path = os.path.join(home, 'Downloads', 'Loughran-McDonald_10X_DocumentDictionaries_1993-2024.txt')
sp500_csv_path = os.path.join(home, "Desktop", 'FIN 377', 'Final-Project-HPST', 'sp500.csv')

# --- Constants ---
target_form = '10-K'
vocab_size = 100000
current_year = 2024
earliest_year = current_year - 20  # Only keep 2004 and newer

# --- Step 2: Load S&P 500 CIKs and Symbols ---
sp500_df = pd.read_csv(sp500_csv_path)
sp500_df['CIK'] = sp500_df['CIK'].astype(str).str.lstrip('0').str.upper()  # Remove leading zeros and capitalize
sp500_ciks = sp500_df['CIK'].tolist()

# Make a lookup dictionary: CIK → Symbol
cik_to_symbol = dict(zip(sp500_df['CIK'], sp500_df['Symbol']))

# --- Functions ---
def parse_word_counts(wordcount_part):
    counts = {}
    for pair in wordcount_part.strip().split(','):
        if ':' in pair:
            try:
                seq, count = map(int, pair.split(':'))
                counts[seq] = count
            except ValueError:
                continue
    return counts

def filing_to_vector(word_counts, vocab_size):
    vector = np.zeros(vocab_size)
    for seq, count in word_counts.items():
        if seq < vocab_size:
            vector[seq] = count
    return vector

# --- Step 3: Read 10-K data and collect filings ---
all_filings = {}  # {CIK: [(period_end, filing_date, word_counts), ...]}

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            header_part, wordcount_part = line.strip().split('|', 1)
            fields = header_part.split(',')
            CIK = fields[0].lstrip('0').upper()
            filing_date = fields[1]  # Get the filing date
            form_type = fields[4]
            if CIK in sp500_ciks and form_type == target_form:
                period_end = int(fields[3])
                word_counts = parse_word_counts(wordcount_part)
                if CIK not in all_filings:
                    all_filings[CIK] = []
                all_filings[CIK].append((period_end, filing_date, word_counts))
        except Exception:
            continue

# --- Step 4: Process filings and calculate cosine distances ---
final_results = []

for CIK in sp500_ciks:
    if CIK not in all_filings:
        continue

    filings = all_filings[CIK]
    filings.sort(key=lambda x: x[0])  # sort by period_end

    previous_vector = None
    previous_filing_date = None

    for period_end, filing_date, word_counts in filings:
        year = int(str(period_end)[:4])
        current_vector = filing_to_vector(word_counts, vocab_size)

        if previous_vector is not None:
            cos_sim = cosine_similarity([previous_vector], [current_vector])[0][0]
            cos_distance = 1 - cos_sim

            if year >= earliest_year:
                final_results.append({
                    'Symbol': cik_to_symbol.get(CIK, 'Unknown'),
                    'CIK': CIK,
                    'Year': year,
                    'Filing Date': filing_date,
                    'Cosine Distance': cos_distance
                })

        previous_vector = current_vector
        previous_filing_date = filing_date

# --- Step 5: Create a DataFrame ---
results_df = pd.DataFrame(final_results)

# Format Filing Date into a datetime
results_df['Filing Date'] = pd.to_datetime(results_df['Filing Date'], format='%Y%m%d')

# Create a new column: Filing Month
results_df['Filing Month'] = results_df['Filing Date'].dt.month

# Reformat Filing Date back to just YYYY-MM-DD
results_df['Filing Date'] = results_df['Filing Date'].dt.date

# Optional: format Filing Date into YYYY-MM-DD
results_df['Filing Date'] = pd.to_datetime(results_df['Filing Date'], format='%Y%m%d').dt.date

# --- Step 6: Display or save the final DataFrame ---
print("\n--- Final DataFrame Preview ---")
print(results_df.head())

# Save it to CSV
output_path = os.path.join(home, 'Downloads', 'sp500_cosine_distances.csv')
results_df.to_csv(output_path, index=False)
print(f"\nSaved results to: {output_path}")

# --- Step 7: End timer ---
end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTotal processing time: {elapsed_time:.2f} seconds")


--- Final DataFrame Preview ---
  Symbol    CIK  Year Filing Date  Cosine Distance  Filing Month
0    MMM  66740  2004  2005-02-24         0.004749             2
1    MMM  66740  2005  2006-02-21         0.010077             2
2    MMM  66740  2006  2007-02-26         0.004970             2
3    MMM  66740  2007  2008-02-15         0.006057             2
4    MMM  66740  2008  2009-02-17         0.007102             2

Saved results to: /Users/henrypiotrowski/Downloads/sp500_cosine_distances.csv

Total processing time: 38.60 seconds


In [2]:
import pandas as pd 
crsp_monthly = pd.read_csv('inputs/crsp_data.csv')

crsp_monthly['date'] = pd.to_datetime(crsp_monthly['date'])
crsp_monthly['filing_month'] = crsp_monthly['date'].dt.month
crsp_monthly['filing_YEAR'] = crsp_monthly['date'].dt.year


crsp_monthly

,permno,date,ret,filing_month,filing_YEAR
0,10000,1986-01-31,NaN,1,1986
1,10000,1986-02-28,-25.7143,2,1986
2,10000,1986-03-31,36.5385,3,1986
3,10000,1986-04-30,-9.8592,4,1986
4,10000,1986-05-30,-22.2656,5,1986
...,...,...,...,...,...
4047625,93436,2024-08-30,-7.7391,8,2024
4047626,93436,2024-09-30,22.1942,9,2024
4047627,93436,2024-10-31,-4.5025,10,2024
4047628,93436,2024-11-29,38.1469,11,2024


In [3]:
# Load the permno mapping file
permno_map = pd.read_csv('inputs/permno_cik_map.csv')

# Ensure both ticker columns are uppercase to match
results_df['Symbol'] = results_df['Symbol'].str.upper()
permno_map['tic'] = permno_map['tic'].str.upper()

# Merge on ticker (Symbol <-> tic)
merged_df = results_df.merge(permno_map, left_on='Symbol', right_on='tic', how='left')

# Drop the redundant 'tic' column if desired
merged_df.drop(columns=['tic'], inplace=True)

merged_df['permno'] = merged_df['permno'].astype('Int64')  # keeps NaNs if any

merged_df


,Symbol,CIK,Year,Filing Date,Cosine Distance,Filing Month,cik,permno
0,MMM,66740,2004,2005-02-24,0.004749,2,66740.0,22592
1,MMM,66740,2005,2006-02-21,0.010077,2,66740.0,22592
2,MMM,66740,2006,2007-02-26,0.004970,2,66740.0,22592
3,MMM,66740,2007,2008-02-15,0.006057,2,66740.0,22592
4,MMM,66740,2008,2009-02-17,0.007102,2,66740.0,22592
...,...,...,...,...,...,...,...,...
11929,ZTS,1555280,2019,2020-02-13,0.002250,2,1555280.0,13788
11930,ZTS,1555280,2020,2021-02-16,0.001143,2,1555280.0,13788
11931,ZTS,1555280,2021,2022-02-15,0.000587,2,1555280.0,13788
11932,ZTS,1555280,2022,2023-02-14,0.020338,2,1555280.0,13788


In [4]:
# 1. Extract filing year from the Filing Date
merged_df['Filing Year'] = pd.to_datetime(merged_df['Filing Date']).dt.year

# 2. Make sure everything is Int64 type
merged_df['Filing Month'] = merged_df['Filing Month'].astype('Int64')
merged_df['Filing Year'] = merged_df['Filing Year'].astype('Int64')
merged_df['permno'] = merged_df['permno'].astype('Int64')

crsp_monthly['filing_month'] = crsp_monthly['filing_month'].astype('Int64')
crsp_monthly['filing_YEAR'] = crsp_monthly['filing_YEAR'].astype('Int64')
crsp_monthly['permno'] = crsp_monthly['permno'].astype('Int64')

# 3. Merge using permno + filing_month + filing_year
final_merged = merged_df.merge(
    crsp_monthly,
    left_on=['permno', 'Filing Month', 'Filing Year'],
    right_on=['permno', 'filing_month', 'filing_YEAR'],
    how='left'
)

# 4. Optional cleanup
final_merged.drop(columns=['filing_month', 'filing_YEAR'], inplace=True, errors='ignore')

final_merged

,Symbol,CIK,Year,Filing Date,Cosine Distance,Filing Month,cik,permno,Filing Year,date,ret
0,MMM,66740,2004,2005-02-24,0.004749,2,66740.0,22592,2005,2005-02-28,0.0000
1,MMM,66740,2005,2006-02-21,0.010077,2,66740.0,22592,2006,2006-02-28,1.7869
2,MMM,66740,2006,2007-02-26,0.004970,2,66740.0,22592,2007,2007-02-28,0.3499
3,MMM,66740,2007,2008-02-15,0.006057,2,66740.0,22592,2008,2008-02-29,-0.9416
4,MMM,66740,2008,2009-02-17,0.007102,2,66740.0,22592,2009,2009-02-27,-14.5380
...,...,...,...,...,...,...,...,...,...,...,...
11929,ZTS,1555280,2019,2020-02-13,0.002250,2,1555280.0,13788,2020,2020-02-28,-0.7302
11930,ZTS,1555280,2020,2021-02-16,0.001143,2,1555280.0,13788,2021,2021-02-26,0.6418
11931,ZTS,1555280,2021,2022-02-15,0.000587,2,1555280.0,13788,2022,2022-02-28,-3.0732
11932,ZTS,1555280,2022,2023-02-14,0.020338,2,1555280.0,13788,2023,2023-02-28,0.9124


In [5]:
# Create shifted month and year for +1 month return
final_merged['Next Month'] = final_merged['Filing Month'] + 1
final_merged['Next Year'] = final_merged['Year']

# Handle year rollover (e.g., December to January)
final_merged.loc[final_merged['Next Month'] > 12, 'Next Month'] = 1
final_merged.loc[final_merged['Filing Month'] == 12, 'Next Year'] += 1

final_merged

,Symbol,CIK,Year,Filing Date,Cosine Distance,Filing Month,cik,permno,Filing Year,date,ret,Next Month,Next Year
0,MMM,66740,2004,2005-02-24,0.004749,2,66740.0,22592,2005,2005-02-28,0.0000,3,2004
1,MMM,66740,2005,2006-02-21,0.010077,2,66740.0,22592,2006,2006-02-28,1.7869,3,2005
2,MMM,66740,2006,2007-02-26,0.004970,2,66740.0,22592,2007,2007-02-28,0.3499,3,2006
3,MMM,66740,2007,2008-02-15,0.006057,2,66740.0,22592,2008,2008-02-29,-0.9416,3,2007
4,MMM,66740,2008,2009-02-17,0.007102,2,66740.0,22592,2009,2009-02-27,-14.5380,3,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11929,ZTS,1555280,2019,2020-02-13,0.002250,2,1555280.0,13788,2020,2020-02-28,-0.7302,3,2019
11930,ZTS,1555280,2020,2021-02-16,0.001143,2,1555280.0,13788,2021,2021-02-26,0.6418,3,2020
11931,ZTS,1555280,2021,2022-02-15,0.000587,2,1555280.0,13788,2022,2022-02-28,-3.0732,3,2021
11932,ZTS,1555280,2022,2023-02-14,0.020338,2,1555280.0,13788,2023,2023-02-28,0.9124,3,2022


In [6]:
final_merged1 = final_merged.merge(
    crsp_monthly,
    how='left',
    left_on=['permno', 'Next Month', 'Next Year'],
    right_on=['permno', 'filing_month', 'filing_YEAR'],
    validate='m:1'  # or 'm:m' depending on your CRSP structure
)

final_merged1

,Symbol,CIK,Year,Filing Date,Cosine Distance,Filing Month,cik,permno,Filing Year,date_x,ret_x,Next Month,Next Year,date_y,ret_y,filing_month,filing_YEAR
0,MMM,66740,2004,2005-02-24,0.004749,2,66740.0,22592,2005,2005-02-28,0.0000,3,2004,2004-03-31,4.9346,3,2004
1,MMM,66740,2005,2006-02-21,0.010077,2,66740.0,22592,2006,2006-02-28,1.7869,3,2005,2005-03-31,2.0848,3,2005
2,MMM,66740,2006,2007-02-26,0.004970,2,66740.0,22592,2007,2007-02-28,0.3499,3,2006,2006-03-31,2.8537,3,2006
3,MMM,66740,2007,2008-02-15,0.006057,2,66740.0,22592,2008,2008-02-29,-0.9416,3,2007,2007-03-30,3.1722,3,2007
4,MMM,66740,2008,2009-02-17,0.007102,2,66740.0,22592,2009,2009-02-27,-14.5380,3,2008,2008-03-31,0.9566,3,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11929,ZTS,1555280,2019,2020-02-13,0.002250,2,1555280.0,13788,2020,2020-02-28,-0.7302,3,2019,2019-03-29,6.8343,3,2019
11930,ZTS,1555280,2020,2021-02-16,0.001143,2,1555280.0,13788,2021,2021-02-26,0.6418,3,2020,2020-03-31,-11.6640,3,2020
11931,ZTS,1555280,2021,2022-02-15,0.000587,2,1555280.0,13788,2022,2022-02-28,-3.0732,3,2021,2021-03-31,1.4429,3,2021
11932,ZTS,1555280,2022,2023-02-14,0.020338,2,1555280.0,13788,2023,2023-02-28,0.9124,3,2022,2022-03-31,-2.6130,3,2022


In [7]:
# Rename columns
final_cleaned = (
    final_merged1
    .rename(columns={
        'date_x': 'Return Date',
        'ret_x': 'Return',
        'ret_y': 'Return +1'
    })
    .loc[:, ['Symbol', 'CIK', 'Year', 'Filing Date', 'Cosine Distance', 'Return Date', 'Return', 'Return +1']]
)

final_cleaned

import os

# Define the output path to Downloads
home = os.path.expanduser('~')
output_path = os.path.join(home, 'Downloads', 'final_cleaned_output.csv')


final_cleaned = final_cleaned.drop_duplicates(subset=['CIK', 'Year'])





In [8]:
missing_rows = final_cleaned[final_cleaned.isna().any(axis=1)]
print(missing_rows)

      Symbol      CIK  Year Filing Date  Cosine Distance Return Date  Return  \
20       AOS    91142  2004  2005-02-18         0.003626         NaT     NaN   
22       AOS    91142  2005  2006-02-24         0.018734         NaT     NaN   
24       AOS    91142  2006  2007-02-16         0.024021         NaT     NaN   
26       AOS    91142  2007  2008-02-20         0.003513         NaT     NaN   
28       AOS    91142  2008  2009-02-25         0.030123         NaT     NaN   
...      ...      ...   ...         ...              ...         ...     ...   
11918    ZBH  1136869  2019  2020-02-21         0.002311         NaT     NaN   
11919    ZBH  1136869  2020  2021-02-22         0.033762         NaT     NaN   
11920    ZBH  1136869  2021  2022-02-25         0.024624         NaT     NaN   
11921    ZBH  1136869  2022  2023-02-24         0.017778         NaT     NaN   
11922    ZBH  1136869  2023  2024-02-23         0.014906         NaT     NaN   

       Return +1  
20           NaN  
2

In [9]:
final_cleaned_ = final_cleaned.dropna()
final_cleaned_ = final_cleaned.dropna().copy()
final_cleaned_['Cosine Similarity'] = 1 - final_cleaned_['Cosine Distance']
final_cleaned_







# Create 5 bins per year based on Cosine Similarity
final_cleaned_['Bins'] = (
    final_cleaned_
    .groupby('Year')['Cosine Similarity']
    .transform(lambda x: pd.qcut(x, q=5, labels=[1, 2, 3, 4, 5], duplicates='drop'))
)



final_cleaned_.to_csv(output_path, index=False)
print(f"Saved file to: {output_path}")

final_cleaned_

Saved file to: /Users/henrypiotrowski/Downloads/final_cleaned_output.csv


,Symbol,CIK,Year,Filing Date,Cosine Distance,Return Date,Return,Return +1,Cosine Similarity,Bins
0,MMM,66740,2004,2005-02-24,0.004749,2005-02-28,0.0000,4.9346,0.995251,5
1,MMM,66740,2005,2006-02-21,0.010077,2006-02-28,1.7869,2.0848,0.989923,3
2,MMM,66740,2006,2007-02-26,0.004970,2007-02-28,0.3499,2.8537,0.995030,4
3,MMM,66740,2007,2008-02-15,0.006057,2008-02-29,-0.9416,3.1722,0.993943,3
4,MMM,66740,2008,2009-02-17,0.007102,2009-02-27,-14.5380,0.9566,0.992898,4
...,...,...,...,...,...,...,...,...,...,...
11929,ZTS,1555280,2019,2020-02-13,0.002250,2020-02-28,-0.7302,6.8343,0.997750,5
11930,ZTS,1555280,2020,2021-02-16,0.001143,2021-02-26,0.6418,-11.6640,0.998857,5
11931,ZTS,1555280,2021,2022-02-15,0.000587,2022-02-28,-3.0732,1.4429,0.999413,5
11932,ZTS,1555280,2022,2023-02-14,0.020338,2023-02-28,0.9124,-2.6130,0.979662,2
